In [1]:
%load_ext autoreload 
%autoreload 2

import operator
import sys

import matplotlib.pyplot as plt
import numpy as np

sys.path.insert(0, '../') # The following modules are in the directory above
from cohort_model import (
    run_cohort_simulation,
    HYP_WILDTYPE, 
    MUT_CAPTIVITY,
)
from figures import ROOT_DIR

# Parameter estimation for case study 1 (*Frontinella pyramitela*)

## Estimating $\alpha$ and $\kappa$

**Note:** The $\alpha$ and $\kappa$ values used in figures 1 and 2 were obtained during an earlier least squares fitting. In restructuring the code for publication we have also improved the fitting procedure and obtained new and better values. However, these are not used in the figures in order to produce results matching those in the published paper (which could not be updated by the time we obtained the better fit).

In [2]:
# Simulation parameters
individual_count = 1000 
repetition_count = 100  

t_m_captivity = 202  # Max t for curve fitting (last x data point = 201) 
t_m_wt = 101         # Max t for curve fitting (last x data point = 100)     

In [3]:
captivity_x = np.genfromtxt(f'{ROOT_DIR}/data/austad_1989/captivity_x.txt')
captivity_y = np.genfromtxt(f'{ROOT_DIR}/data/austad_1989/captivity_y.txt')

xdata = np.round(captivity_x).astype('int64') # In order to use for indexing
ydata = captivity_y * individual_count

In [ ]:
%%time
data_count = len(xdata)

fit = []
# TODO: The population is set to mutant captivity in order to 
for alpha in np.arange(0.0002, 0.0003, 0.00001): 
    for kappa in np.arange(0.02, 0.04, 0.0001): 
        hazard_rate_params = dict(alpha=alpha, kappa=kappa, population=MUT_CAPTIVITY)
        population_survivorship = run_cohort_simulation(
            repetition_count, 
            individual_count, 
            hazard_rate_params, 
            t_m_captivity
        )
        mean = np.mean(population_survivorship, axis=0)[xdata]
        squares = [(mean[index] - ydata[index])**2 for index in range(data_count)] 
        fit.append((alpha, kappa, sum(squares)))

In [ ]:
best_fits = sorted(fit, key=operator.itemgetter(2))
print(*best_fits[0:10], sep='\n')

### Estimating $\epsilon$ and $h_{wt}(t)$

In [ ]:
x_wild = np.genfromtxt(f'{ROOT_DIR}/data/austad_1989/wild_x.txt')
y_wild = np.genfromtxt(f'{ROOT_DIR}/data/austad_1989/wild_y.txt')

xdata_w = np.round(x_wild).astype('int64') # In order to use for indexing
ydata_w = y_wild * individual_count

xdata_w = xdata_w[:-2] # In order not to fit to the last two data points
ydata_w = ydata_w[:-2] # In order not to fit to the last two data points

In [ ]:
%%time
n = len(xdata_w)

fit = []
# TODO: The population is set to the hypothetical wild type, in order to use the 
for prod_wt in np.arange(0.0378, 0.0382, 0.000025):  # prod_wt = (1 - epsilon) * h_wt
    hazard_rate_params = dict(hazard_rate_wild_type=prod_wt, population=HYP_WILDTYPE)
    population_survivorship = run_cohort_simulation(
        repetition_count, 
        cohort, 
        hazard_rate_params, 
        t_m_wt,
    )
    mean = np.mean(population_survivorship, axis=0)[xdata_w]
    squares = [(mean[i] - ydata_w[i])**2 for i in range(n)] # Not fitting to last two data points
    fit.append((prod_wt, sum(squares)))

In [ ]:
best_fits = sorted(fit, key=operator.itemgetter(1))
print(*best_fits[0:10], sep='\n')

In [ ]:
prod_wt = best_fits[0][0]
for epsilon in np.arange(0.01, 0.05, 0.01):
    h_wt = prod_wt / (1 - epsilon)
    print(f'epsilon = {epsilon}, h_wt = {h_wt}')